In [3]:
#STEP1_PREPROCESS_2condition_vs_2condition_linear_scatter_plot_with_CohensD
#
#Preprocess for creating 2condition vs. 2condition scatter plot.
#where x-axis is log2fc(condition_b / condition_a)
#where y-axis is log2fc(condition_c / condition_a)

import pandas as pd
import statistics
import math

In [5]:
#x-axis = foldchange (control vs. ACPA+)
#y-axis = foldchange (control vs. ACPA+)
#color coding scheme = significance at A or B

#color 1 = significant at set1
#color 2 = significant at set2
#color 3 = significant at set1 & set 2

In [8]:
def get_data_dict(data_file, data_dict, common_feature_list, comparison_type):
       
    data_df = pd.read_csv(data_file,sep='\t', index_col=0)
    feature_list = data_df.index.values
    r, c = data_df.shape

    for i in range(r):
        
        feature = feature_list[i]
        pval = float(data_df["pval"][i])
        fc = float(data_df["fc_case_control"][i])
        cohenD = float(data_df["cohenD"][i])
        
        data_dict[comparison_type, feature] = [fc, pval, cohenD]
        
        if feature not in common_feature_list:
            common_feature_list.append(feature)
        #technically, 'common_feature_list' should be same as 'feature_list'

    return data_dict, feature_list

def make_output(omics, data_dict, feature_list):
    
    output_dir = '../../../analysis/statistics/scatter_plots/data'
    
    output_file = '%s/%s.plot_ready.cohenD.tsv' % (output_dir, omics)
    output_txt = open(output_file,'w')
    
    output_txt.write('\tcVSneg_FC\tcVSpos_FC\tsig_type\n')
    print ('omics: %s'% omics)
    for feature in feature_list:
        
        condition_a_fc = data_dict['cVSneg', feature][0]
        condition_a_pval = data_dict['cVSneg', feature][1]
        condition_a_cohenD = data_dict['cVSneg', feature][2]
        
        condition_b_fc = data_dict['cVSpos', feature][0]
        condition_b_pval = data_dict['cVSpos', feature][1]
        condition_b_cohenD = data_dict['cVSpos', feature][2]
            
        sig_type = 0
        
        if omics != "autoantibody": #not autoantibody
            if condition_a_pval < 0.01 and condition_b_pval >= 0.01:
                if condition_a_cohenD > 0.5:
                    sig_type = 1
            if condition_a_pval >= 0.01 and condition_b_pval < 0.01:
                if condition_b_cohenD > 0.5:
                    sig_type = 2
            if condition_a_pval < 0.01 and condition_b_pval < 0.01:
                if condition_a_cohenD > 0.5 and condition_b_cohenD < 0.5:
                    sig_type = 1                    
                if condition_a_cohenD < 0.5 and condition_b_cohenD > 0.5:
                    sig_type = 2                    
                if condition_a_cohenD > 0.5 and condition_b_cohenD > 0.5:
                    sig_type = 3
                    
        if omics == "autoantibody": #is autoantibody
            if condition_a_pval < 0.05 and condition_b_pval >= 0.05:
                if condition_a_cohenD > 0.5:
                    sig_type = 1
            if condition_a_pval >= 0.05 and condition_b_pval < 0.05:
                if condition_b_cohenD > 0.5:
                    sig_type = 2
            if condition_a_pval < 0.05 and condition_b_pval < 0.05:

                if condition_a_cohenD > 0.5 and condition_b_cohenD < 0.5:
                    sig_type = 1                    
                if condition_a_cohenD < 0.5 and condition_b_cohenD > 0.5:
                    sig_type = 2                    
                if condition_a_cohenD > 0.5 and condition_b_cohenD > 0.5:
                    sig_type = 3

        output_txt.write('%s\t%s\t%s\t%s\n' % (feature, condition_a_fc, condition_b_fc, sig_type))

    output_txt.close()
    
    

In [9]:
#main
diff_dir = '../../../analysis/statistics/linear_model/differential_abundance'
omics_list = ['autoantibody', 'metabolomics', 'proteomics']

for omics in omics_list:

    data_dict = {}
    feature_list = []

    cVSneg_data_file = '%s/%s.cVSneg.padj.v2.tsv' % (diff_dir, omics)
    cVSpos_data_file = '%s/%s.cVSpos.padj.v2.tsv' % (diff_dir, omics)
    
    data_dict, feature_list = get_data_dict(cVSneg_data_file, data_dict, feature_list, 'cVSneg')
    data_dict, feature_list = get_data_dict(cVSpos_data_file, data_dict, feature_list, 'cVSpos')
    
    print (omics)
    print (len(feature_list))

    #write outputx
    make_output(omics, data_dict, feature_list)

    

autoantibody
1610
omics: autoantibody
metabolomics
1061
omics: metabolomics
proteomics
7272
omics: proteomics
